In [2]:
import numpy as np
import cv2

In [3]:
img=cv2.imread('./Project_5_Automatic_Face_Blur/Project_Automatic_Face_Blur/images/happy_parents.jpg')

cv2.imshow('Parents',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
# Load the face detection model 
face_detection_model=cv2.dnn.readNetFromCaffe('./Project_5_Automatic_Face_Blur/Project_Automatic_Face_Blur/model/deploy.prototxt.txt',
                                           './Project_5_Automatic_Face_Blur/Project_Automatic_Face_Blur/model/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [24]:
def face_blur(img):
    #blob from image
    image=img.copy()

    # create a mask
    face_mask=np.zeros(image.shape[:2],dtype='uint8')
    blob=cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=True)
    face_detection_model.setInput(blob)
    detections=face_detection_model.forward()

    h,w =image.shape[:2]
    for i in range(detections.shape[2]):
            confidence=detections[0,0,i,2]
            if confidence>0.5:
                box=detections[0,0,i,3:7]*np.array([w,h,w,h])
                box=box.astype(int)
                cv2.rectangle(face_mask,(box[0],box[1]),(box[2],box[3]),255,-1)

    inverse_face_mask=cv2.bitwise_not(face_mask)   
    # apply mask to orginal image

    bg_img=cv2.bitwise_and(image,image,mask=face_mask)
    fg_img=cv2.bitwise_and(image,image,mask=inverse_face_mask)

    #apply blur to image
    bg_img_blur=cv2.GaussianBlur(bg_img,(51,51),0)

    # addition
    face_blur=cv2.add(fg_img,bg_img_blur)
    
    return face_blur
                            

In [25]:
face_blur=face_blur(image)

cv2.imshow('Orginal image',image)
cv2.imshow('face_blur',face_blur)
cv2.waitKey()
cv2.destroyAllWindows() 
                     

## Real time face blur

In [32]:
cap=cv2.VideoCapture(0)

while True:
    ret, frame=cap.read()
    
    if ret == False:
        break
        
    blur_image=face_blur(frame)
    
    cv2.imshow('Orginal',frame)
    cv2.imshow('blur_image',blur_image)
    
    if cv2.waitKey(1)==27:
        break
        
cap.release()
cv2.destroyAllWindows()
    

TypeError: 'numpy.ndarray' object is not callable

In [6]:
detections.shape

(1, 1, 200, 7)